# Workshop - Data Science & Data Mining
###### NEREUS-USP, São Paulo, April 22, 2019


### *Leonardo Caio de Ladalardo Martins*
lcladalardo@gmail.com

_*Undergraduate student at FEA-USP_

## Section 5: Data Mining - Applications

This case study focuses on the CNPJ Public Database, available at [Receita Federal](http://receita.economia.gov.br/orientacao/tributaria/cadastros/cadastro-nacional-de-pessoas-juridicas-cnpj/dados-publicos-cnpj). This database is very rich in terms of details, but really heavy as well, it contains about:
1. 5gb of data (compressed) and almost 90gb of pure data (not compressed);
2. More than 140mn of rows of diferent layout tables;
3. Personal information about all firms (PJ) and partners (PF) in Brazil;
4. An impressive amount of geographic information about firms (but requiring to be cleaned as well);
5. Almost 70mn of entries overall formats;

In [2]:
#Loading important libraries
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sb
import time 
import numpy as np
import glob

## Section 5.1: A Case Study - Reading the CNPJ Database

The aim of this section is to provide an example how to load real data. As we're going to see, 'Real Data is messy', but we need to create a strategy to deal with the potential issues that are going to arrive. 

#### What we know taking a quick glance in the dataset?
1) It's a fixed width file - it will require a dictionary providing the width of each field;

2) It has three different kinds of table delimited by a "Format Column" - this will be hugely important while loading the algorithm and processing the data

3) We'll use Pandas to load data - it will require some adaptations to be computationally efficient;

### Step 1: Selecting what we want to load - use parsimony!
1. Select the variables that we want to open - do we really need to open the whole file?
2. Create a name for those selected variables;
3. Be carefull with long and heavy string variables;

In [5]:
#Width of the columns of table1:
colspecs = [(0,1),(3,17),(17,18),(367,375),
            (375,382),(682,684),(684,688),(688,738)]

In [6]:
#Name of the columns of table 1:
col_nam = ['TIPO DE REGISTRO','CNPJ', 'IDENTIFICADOR MATRIZ/FILIAL',
             'DATA INICIO ATIVIDADE','CEP', 'UF', 'CODIGO MUNICIPIO',
            'MUNICIPIO']

### Step 2: Creating the algorithm - efficiency is key over here
1. Initialize the algorithm based on its default values, dictionaries and also on the **chunksize** - this last item is very important to load data progressively allowing a fractionalization of the dataset;
2. Test the number of chunks on 0dataset - we've adopted about 1mn of rows for each subset of data, it will depends on your CPU and GPU power;
3. Put all data together based on a one-by-rest scheme - iterator of the algo - this will define the 'order' of your solution; 
4. Creating time-estimation variables - this would be interesting to preview the total time to handle with all data at once - your algorithm is trully efficient?

In [7]:
class table:
    def __init__(self, path):
        self.data = pd.read_fwf(path, colspecs, iterator = True, chunksize = 10**6, dtype='str', header = None)
            
    def test(self):
        for chunk in self.data:
            print(len(chunk))
            
    def table(self):
        self.df = pd.DataFrame()
        self.data = pd.read_fwf(path, colspecs, iterator = True, chunksize = 10**6, dtype='str', header = None)
        i = 0
        n = 0
        chunk_start = time.time()
        #test:
        for chunk in self.data:
            chunk.columns = col_nam
            chunk = chunk[(chunk["TIPO DE REGISTRO"]).astype(int) == 1]
            self.df = pd.concat([self.df, chunk])
            #Time elapsed:
            chunk_end = time.time()
            time_elapsed = chunk_end-chunk_start
            hours, rem = divmod(time_elapsed, 3600)
            minutes, seconds = divmod(rem, 60)
            #Total time (estimated):
            total_time = (time_elapsed/(i+1))*(72-i)
            hours2, rem2 = divmod(total_time, 3600)
            minutes2, seconds2 = divmod(rem2, 60)
            print('Chunk: ' + str(i) +" ; Time elapsed: " + "{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds) +
                 " ; Total time (estimated): " + "{:0>2}:{:0>2}:{:05.2f}".format(int(hours2),int(minutes2),seconds2))
            #Counters
            i += 1
            n += 1
            #Testing
        end3 = time.time()
        hours3, rem3 = divmod(end3-chunk_start, 3600)
        minutes3, seconds3 = divmod(rem3, 60)
        print("Total time: " + "{:0>2}:{:0>2}:{:05.2f}".format(int(hours3),int(minutes3),seconds3))
        return self.df

In [8]:
path = 'E:/Dados Empresa/DADOS_ABERTOS_CNPJ/cnpj.txt'

In [11]:
#Initializing our algorithm with its default values:
tab = table(path)

In [12]:
#Creating the table in formats 1:
table = tab.table()

Chunk: 0 ; Time elapsed: 00:00:20.18 ; Total time (estimated): 00:24:13.07
Chunk: 1 ; Time elapsed: 00:01:00.58 ; Total time (estimated): 00:35:50.69
Chunk: 2 ; Time elapsed: 00:01:40.78 ; Total time (estimated): 00:39:11.43
Chunk: 3 ; Time elapsed: 00:02:21.38 ; Total time (estimated): 00:40:38.81
Chunk: 4 ; Time elapsed: 00:03:01.68 ; Total time (estimated): 00:41:10.91
Chunk: 5 ; Time elapsed: 00:03:42.45 ; Total time (estimated): 00:41:24.03
Chunk: 6 ; Time elapsed: 00:04:22.97 ; Total time (estimated): 00:41:19.41
Chunk: 7 ; Time elapsed: 00:05:03.89 ; Total time (estimated): 00:41:09.09
Chunk: 8 ; Time elapsed: 00:05:45.12 ; Total time (estimated): 00:40:54.18
Chunk: 9 ; Time elapsed: 00:06:26.03 ; Total time (estimated): 00:40:31.96
Chunk: 10 ; Time elapsed: 00:07:07.12 ; Total time (estimated): 00:40:07.39
Chunk: 11 ; Time elapsed: 00:07:48.11 ; Total time (estimated): 00:39:39.58
Chunk: 12 ; Time elapsed: 00:08:29.72 ; Total time (estimated): 00:39:12.57
Chunk: 13 ; Time elaps

### Step 3: Final Result - this is what you really need?

Now you've done all the dirty job to put all data together. However, it will require some extra manipulation to extract the relevant information for our analysis. This will be our aim for the next sections.

In [13]:
table

,TIPO DE REGISTRO,CNPJ,IDENTIFICADOR MATRIZ/FILIAL,DATA INICIO ATIVIDADE,CEP,UF,CODIGO MUNICIPIO,MUNICIPIO
1,1,00000000000191,1,19660801,6422100,DF,9701,BRASILIA
40,1,00000000000272,2,19760813,6421200,AM,0255,MANAUS
41,1,00000000000353,2,19660801,6422100,PA,0427,BELEM
42,1,00000000000434,2,19660801,6422100,SP,7071,SANTOS
43,1,00000000000515,2,19660801,6421200,RJ,5819,CAMPOS DOS GOYTACAZES
44,1,00000000000604,2,19660801,6422100,BA,3849,SALVADOR
45,1,00000000000787,2,19660801,6422100,PE,2531,RECIFE
46,1,00000000000868,2,19660801,6422100,CE,1389,FORTALEZA
47,1,00000000000949,2,19660801,6421200,PR,7535,CURITIBA
48,1,00000000001082,2,19660801,6422100,RS,8801,PORTO ALEGRE


## Section 5.2 - Manipulating your Dataset

### Section 5.2.1. Creating a summary table mapping the number of firms born over each county per year
Now, our objective is to create a summary table for our dataset. We're going to keep the most important concept in Data Mining: be objective! Keep only what you need - this would be easier while manipulating the data. 

#### Steps:
1. Keep only the variables that you're going to use on this table;
2. Clean your string variables;
3. Transform time and data format to your purpouses;
4. Develop a solution to pivot your sub-dataset;

In [18]:
df = table[['DATA INICIO ATIVIDADE', 'MUNICIPIO']]

In [24]:
# Step 1:
df = table[['DATA INICIO ATIVIDADE', 'MUNICIPIO']]

# Step 2:
df['MUNICIPIO'] = df['MUNICIPIO'].str.replace('\d+', '')

# Step 3:
def time_conversor(name, col, df):
    df[name] = pd.to_datetime(df[col], errors = 'coerce').dt.year
    df = df[pd.notnull(df['DATA'])]    
    df[name] = df[name].astype(int)
    return df
# Converting the time format:
df = time_conversor('DATA', 'DATA INICIO ATIVIDADE', df)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas

In [25]:
# Step 4: manipulate your dataset using pandas' functionalities
class data_visualization:
    def __init__(self, df):
        self.df = df
    
    def frequency(self, ind, col, val):
        self.ind = ind
        self.col = col
        self.val = val
        self.freq = self.df.groupby([self.ind, self.col]).size().reset_index(name = self.val)
        return self.freq, self.ind, self.col, self. val
    
    def pivot(self, ind, col, val):
        self.freq, self.ind, self.col, self. val = self.frequency(ind, col, val)
        self.data = self.freq.reset_index()
        self.pivot = self.data.pivot_table(index = self.ind, columns = self.col, values = self.val, 
                                           aggfunc = 'sum', margins = True)
        self.pivot = self.pivot.fillna(0)
        return self.pivot

In [26]:
# Initializing algorithm
data = data_visualization(df)
# Creating the pivot-table:
database = data.pivot('DATA', 'MUNICIPIO', 'FREQ')

In [27]:
database

MUNICIPIO,ABADIA DE GOIAS,ABADIA DOS DOURADOS,ABADIANIA,ABAETE,ABAETETUBA,ABAIARA,ABAIRA,ABARE,ABATIA,ABDON BATISTA,...,XAVANTINA,XAXIM,XEXEU,XINGUARA,XIQUE-XIQUE,ZABELE,ZACARIAS,ZE DOCA,ZORTEA,All
DATA,,,,,,,,,,,,,,,,,,,,,
1696,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1891,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1893,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1899,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1901,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,159
1902,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1903,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8
1904,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
1905,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4


In [28]:
database.describe()

MUNICIPIO,ABADIA DE GOIAS,ABADIA DOS DOURADOS,ABADIANIA,ABAETE,ABAETETUBA,ABAIARA,ABAIRA,ABARE,ABATIA,ABDON BATISTA,...,XAVANTINA,XAXIM,XEXEU,XINGUARA,XIQUE-XIQUE,ZABELE,ZACARIAS,ZE DOCA,ZORTEA,All
count,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,...,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,1.230000e+02
mean,34.422764,17.788618,40.065041,88.373984,162.211382,8.780488,12.292683,18.048780,20.617886,7.349593,...,13.853659,103.642276,14.032520,103.479675,63.902439,4.146341,22.032520,56.585366,11.512195,6.380313e+05
std,194.991005,99.467455,225.338315,491.433791,909.346002,49.269672,68.580183,101.619013,115.031750,41.444409,...,77.184756,576.333509,79.613506,579.006970,356.360138,23.776860,125.370267,316.301525,64.882032,3.557346e+06
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+00
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000e+00
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000e+02
75%,4.000000,11.000000,20.500000,70.000000,75.500000,4.000000,9.000000,5.000000,14.000000,2.000000,...,11.500000,90.500000,1.000000,79.500000,50.500000,0.000000,2.000000,45.500000,1.000000,5.594385e+05
max,2117.000000,1094.000000,2464.000000,5435.000000,9976.000000,540.000000,756.000000,1110.000000,1268.000000,452.000000,...,852.000000,6374.000000,863.000000,6364.000000,3930.000000,255.000000,1355.000000,3480.000000,708.000000,3.923892e+07


### Do you notice something wrong on this dataset?
- The first firm established in Brazil was Santanense in 1891 - but why we have a firm in 1696?
- Why in 1901 we have a huge jump in terms of firms?

**Lesson 1: Real Data is not as perfect as we would wonder...**

In [69]:
#We're going to discard this strange observations
data = database.iloc[5:,:]

### Section 5.2.2: Adding some new features on the dataset

What if we add some geo-locational reference to the Brazilian counties?

As an example, I have merged CNPJ dataset with "Municipios do Brasil com Latitude e Longitude" available at [Qlik Brasil](https://community.qlik.com/t5/Qlik-Brasil/Municipios-do-Brasil-com-Latitude-e-Longitude/gpm-p/1480217). On the next notebook, I will plot this geo-referenced data on Brazil's map with the purpouse to introduce some data visualization libraries disposable in Python.

In [76]:
#Loading the auxiliar dataset
aux = pd.read_excel('d:/Users/leonardo.martins/Documents/Workshop - NEREUS/MunicipiosBrasil.xls')

In [77]:
# Initializing algorithm - the same as developed before
data2 = data_visualization(df_new)
# Creating the pivot-table:
data_new, _, _, _, = data2.frequency('DATA', 'MUNICIPIO', 'FREQ')

In [78]:
#Merging both datasets on the pre-processed data:
data_new = pd.merge(data_new, aux, how='left', left_on='MUNICIPIO',right_on='MUNICIPIO')

In [79]:
data_new

,DATA,MUNICIPIO,FREQ,LONGITUDE,LATITUDE,UF
0,1696,JUIZ DE FORA,1,-43.35,-21.76,MG
1,1891,MONTES CLAROS,1,-43.86,-16.73,MG
2,1893,BELO HORIZONTE,1,-43.95,-19.81,MG
3,1899,SAO PAULO,1,-46.63,-23.54,SP
4,1901,ABRE CAMPO,1,-42.47,-20.30,MG
5,1901,ALAGOINHA,2,-35.54,-6.95,PB
6,1901,ALAGOINHA,2,-36.77,-8.46,PE
7,1901,ALEGRETE,1,-55.79,-29.78,RS
8,1901,ALMENARA,1,-40.69,-16.18,MG
9,1901,ALTINHO,1,-36.05,-8.49,PE


### Final Result: Data Tables

Now that we've both datasets, we're going to export them as xlsx to both save server space, and also allowing a more efficient computational processing.

In [75]:
data.to_excel('d:/Users/leonardo.martins/Documents/Workshop - NEREUS/dataset1.xlsx')
data_new.to_excel('d:/Users/leonardo.martins/Documents/Workshop - NEREUS/dataset2.xlsx')